In [1]:
# Author: Emanuele Zeppieri <emazep@gmail.com>
#
# This code is distributed under the terms and conditions
# from the MIT License (MIT).

In [2]:
import datetime as dt
from datetime import date, datetime, timedelta
import time
import os.path
import tabula

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
ORIGINAL_DOC_DIR = '../original_ISS_documents/bollettino_sorveglianza_integrata/'
FILE_PREFIX = 'Bollettino-sorveglianza-integrata-COVID-19_'
EXPORT_DIR = '../data/'
FILES_TO_EXCLUDE = ['Bollettino-sorveglianza-integrata-COVID-19_2020-03-09.pdf']

# Let Java wait this number of seconds in case of tabula (Java) timing issues.
WAIT = 1

In [4]:
COLUMNS_CASES_MALE = [
    'cases_male_0-9', 'cases_male_10-19', 'cases_male_20-29',
    'cases_male_30-39', 'cases_male_40-49', 'cases_male_50-59',
    'cases_male_60-69', 'cases_male_70-79', 'cases_male_80-89',
    'cases_male_90-'
]
COLUMNS_DEATHS_MALE = [
    'deaths_male_0-9', 'deaths_male_10-19', 'deaths_male_20-29',
    'deaths_male_30-39', 'deaths_male_40-49', 'deaths_male_50-59',
    'deaths_male_60-69', 'deaths_male_70-79', 'deaths_male_80-89',
    'deaths_male_90-'
]
COLUMNS_CASES_FEMALE = [
    'cases_female_0-9', 'cases_female_10-19', 'cases_female_20-29',
    'cases_female_30-39', 'cases_female_40-49', 'cases_female_50-59',
    'cases_female_60-69', 'cases_female_70-79', 'cases_female_80-89',
    'cases_female_90-'
]
COLUMNS_DEATHS_FEMALE = [
    'deaths_female_0-9', 'deaths_female_10-19', 'deaths_female_20-29',
    'deaths_female_30-39', 'deaths_female_40-49', 'deaths_female_50-59',
    'deaths_female_60-69', 'deaths_female_70-79', 'deaths_female_80-89',
    'deaths_female_90-'
]

In [5]:
table_pages = {
    'Bollettino-sorveglianza-integrata-COVID-19_2020-12-22.pdf': 23,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-12-16.pdf': 23,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-11-25.pdf': 23,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-10-13.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-10-06.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-29.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-22.pdf': 20,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-15.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-08.pdf': 20,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-01.pdf': 20,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-25.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-18.pdf': 19,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-11.pdf': 12,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-04.pdf': 10,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-28.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-21.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-14.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-07.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-30.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-23.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-16.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-09.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-03.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-26.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-20.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-14.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-07.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-28.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-23.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-16.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-09.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-06.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-02.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-30.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-26.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-23.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-16.pdf': 5,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-12.pdf': 5,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-09.pdf': 4
}
DEFAULT_TABLE_PAGE = 22

In [6]:
# Utility: to be used with new docs which size is unknown as needed (to fill the corresponding ETL_SPECS entry)
docs = ['Bollettino-sorveglianza-integrata-COVID-19_2020-12-29.pdf']

for file_name in docs:
    if file_name in FILES_TO_EXCLUDE:
        # Skip unusable ISS bulletins.
        continue
    
    print(file_name)
    file = ORIGINAL_DOC_DIR + file_name
    table_page = table_pages.get(file_name, DEFAULT_TABLE_PAGE)    
    df_pdf = tabula.read_pdf(file, pages=table_page)[0]
    df_pdf_norm = pd.DataFrame()
    
    for c in df_pdf.columns:
        df_pdf_norm = pd.concat([df_pdf_norm, df_pdf[c].astype(str).str.split(expand=True)], axis=1)
        df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    # Drop spurious rows
    idx1 = df_pdf_norm[df_pdf_norm[0]=='0-9'].index.values[0]
    idx2 = df_pdf_norm[(df_pdf_norm[0]=='>90') | (df_pdf_norm[0]=='≥90')].index.values[0]
    df_pdf_norm = df_pdf_norm[idx1:idx2+1]
    df_pdf_norm.reset_index(drop=True, inplace=True)

    # Drop spurious columns
    df_pdf_norm.dropna(how='all', axis='columns', inplace=True)
    df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    print(df_pdf_norm.size)
    print(str(len(df_pdf_norm)) + '*' + str(len(df_pdf_norm.columns)))
    print()


df_pdf_norm

Bollettino-sorveglianza-integrata-COVID-19_2020-12-29.pdf
160
10*16



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0-9,40.723,"51,8",5,"55,6",0,37.941,"48,2",4,"44,4",0,78.664,"3,8",9,0,0
1,10-19,88.983,"52,3",6,60,0,81.065,"47,7",4,40,0,170.048,"8,3",10,0,0
2,20-29,124.114,"50,6",19,"54,3",0,121.340,"49,4",16,"45,7",0,245.458,12,35,0,0
3,30-39,123.311,"49,1",94,"62,7","0,1",127.911,"50,9",56,"37,3",0,251.226,"12,3",150,"0,2","0,1"
4,40-49,152.263,"46,6",421,"69,9","0,3",174.307,"53,4",181,"30,1","0,1",326.571,"15,9",602,"0,9","0,2"
5,50-59,178.171,"48,3",1.765,"74,3",1,190.462,"51,7",611,"25,7","0,3",368.635,18,2.376,"3,4","0,6"
6,60-69,122.158,"53,3",5.066,"74,3","4,1",107.041,"46,7",1.752,"25,7","1,6",229.200,"11,2",6.818,"9,6",3
7,70-79,89.991,"52,3",12.058,"68,9","13,4",82.080,"47,7",5.435,"31,1","6,6",172.071,"8,4",17.493,"24,7","10,2"
8,80-89,60.854,"40,6",16.096,"55,1","26,5",89.092,"59,4",13.128,"44,9","14,7",149.953,"7,3",29.224,"41,3","19,5"
9,≥90,13.119,"22,6",4.751,"33,7","36,2",44.869,"77,4",9.329,"66,3","20,8",57.988,"2,8",14.080,"19,9","24,3"


In [7]:
ETL_SPECS = {
    '10*16': {
        'col_cases_male': 1, 'col_deaths_male': 3,
        'col_cases_female': 6, 'col_deaths_female': 8
    },
    '10*17': {
        'col_cases_male': 1, 'col_deaths_male': 3,
        'col_cases_female': 6, 'col_deaths_female': 8
    },
    '10*18': {
        'col_cases_male': 1, 'col_deaths_male': 3,
        'col_cases_female': 6, 'col_deaths_female': 8
    },
    '10*25': {
        'col_cases_male': 1, 'col_deaths_male': 4,
        'col_cases_female': 9, 'col_deaths_female': 12
    }
}

In [8]:
def scraped_df_to_row(
    scraped_df,
    col_cases_male, col_deaths_male,
    col_cases_female, col_deaths_female
):
    row = scraped_df.iloc[:, col_cases_male].tolist()
    row += scraped_df.iloc[:, col_deaths_male].tolist()
    
    row += scraped_df.iloc[:, col_cases_female].tolist()
    row += scraped_df.iloc[:, col_deaths_female].tolist()
    
    return [int(str(v).replace('.', '').replace(',', '').replace('-', '0')) for v in row]

In [9]:
# ETL loop
ds = pd.DataFrame(columns = COLUMNS_CASES_MALE + COLUMNS_DEATHS_MALE + COLUMNS_CASES_FEMALE + COLUMNS_DEATHS_FEMALE)

docs = sorted(os.listdir(ORIGINAL_DOC_DIR))

for file_name in docs:
    if (
        file_name in FILES_TO_EXCLUDE or
        not file_name.startswith(FILE_PREFIX)
    ):
        # Skip unusable ISS bulletins and spurious files.
        continue
    
    print('Scraping')
    print(file_name)
    
    file = ORIGINAL_DOC_DIR + file_name
    table_page = table_pages.get(file_name, DEFAULT_TABLE_PAGE)    
    df_pdf = tabula.read_pdf(file, pages=table_page)[0]
    df_pdf_norm = pd.DataFrame()
    
    for c in df_pdf.columns:
        df_pdf_norm = pd.concat([df_pdf_norm, df_pdf[c].astype(str).str.split(expand=True)], axis=1)
        df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    # Drop spurious rows
    idx1 = df_pdf_norm[df_pdf_norm[0]=='0-9'].index.values[0]
    idx2 = df_pdf_norm[(df_pdf_norm[0]=='>90') | (df_pdf_norm[0]=='≥90')].index.values[0]
    df_pdf_norm = df_pdf_norm[idx1:idx2+1]
    df_pdf_norm.reset_index(drop=True, inplace=True)

    # Drop spurious columns
    df_pdf_norm.dropna(how='all', axis='columns', inplace=True)
    df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    scraped_table_dim = str(len(df_pdf_norm)) + '*' + str(len(df_pdf_norm.columns))
    print(scraped_table_dim)
    
    # Flatten the whole ISS table into a single row.
    doc_date = date.fromisoformat(file_name[-14:-4])
    ds.loc[doc_date] = scraped_df_to_row(df_pdf_norm, **ETL_SPECS[scraped_table_dim])

    print()
    time.sleep(WAIT)

ds

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-03-12.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-03-16.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-03-23.pdf
10*17

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-03-26.pdf
10*17

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-03-30.pdf
10*17

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-04-02.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-04-06.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-04-09.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-04-16.pdf
10*17

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-04-23.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-04-28.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-05-07.pdf
10*16

Scraping
Bollettino-sorveglianza-integrata-COVID-19_2020-05-14.pdf
10*16

Scraping
Bollettino-sorveglianza-integ

,cases_male_0-9,cases_male_10-19,cases_male_20-29,cases_male_30-39,cases_male_40-49,cases_male_50-59,cases_male_60-69,cases_male_70-79,cases_male_80-89,cases_male_90-,deaths_male_0-9,deaths_male_10-19,deaths_male_20-29,deaths_male_30-39,deaths_male_40-49,deaths_male_50-59,deaths_male_60-69,deaths_male_70-79,deaths_male_80-89,deaths_male_90-,cases_female_0-9,cases_female_10-19,cases_female_20-29,cases_female_30-39,cases_female_40-49,cases_female_50-59,cases_female_60-69,cases_female_70-79,cases_female_80-89,cases_female_90-,deaths_female_0-9,deaths_female_10-19,deaths_female_20-29,deaths_female_30-39,deaths_female_40-49,deaths_female_50-59,deaths_female_60-69,deaths_female_70-79,deaths_female_80-89,deaths_female_90-
2020-03-12,36,60,214,423,735,1359,1611,1916,1306,155,0,0,0,1,0,10,53,216,252,40,27,57,272,353,717,1033,744,886,794,230,0,0,0,0,1,4,12,57,103,35
2020-03-16,75,97,419,829,1491,2633,3007,3461,2322,287,0,0,0,4,5,34,113,469,491,81,45,87,517,794,1425,2002,1386,1621,1518,472,0,0,0,0,4,12,29,130,234,84
2020-03-23,181,200,983,1988,3601,6276,6943,7458,4930,678,0,0,0,9,31,129,437,1371,1358,208,135,184,1166,1952,3572,4887,3405,3782,3590,1242,0,0,0,3,10,38,99,390,656,253
2020-03-26,244,261,1203,2465,4597,7998,8755,9309,6195,887,0,0,0,14,49,190,606,1846,1808,273,180,249,1510,2494,4570,6337,4394,4781,4734,1640,0,0,0,3,18,52,154,555,894,334
2020-03-30,324,392,1631,3105,5802,10068,10744,11236,7630,1160,0,0,1,18,66,294,923,2597,2603,424,260,371,2132,3352,6198,8495,5584,6142,6504,2404,0,0,1,2,23,74,236,854,1378,514
2020-04-02,381,475,1919,3505,6480,11149,11761,12281,8553,1374,0,0,4,22,81,377,1144,3146,3259,558,307,454,2531,3885,7130,9706,6254,6868,7644,2973,0,0,2,7,29,100,299,1041,1767,692
2020-04-06,450,621,2426,4158,7444,12648,13057,13581,9868,1651,0,0,4,24,102,447,1349,3657,3793,670,375,596,3181,4780,8701,11608,7306,7990,9525,4024,1,0,3,10,34,118,372,1244,2157,854
2020-04-09,509,717,2752,4612,8021,13469,13784,14234,10661,1894,0,0,5,25,114,504,1532,3968,4255,760,425,712,3576,5316,9676,12864,7908,8666,10935,4941,1,0,2,11,39,133,423,1393,2450,1022
2020-04-16,596,901,3350,5344,9009,14779,14963,15577,12332,2470,0,0,5,28,133,606,1776,4532,4992,970,519,897,4338,6299,11452,15015,9024,10097,14329,7333,1,0,2,12,45,150,504,1664,3071,1483
2020-04-23,687,1070,3877,5951,9849,15772,15825,16410,13446,2888,1,0,5,30,154,681,1993,4952,5642,1210,607,1070,5024,7136,12851,16685,9825,11163,17034,9429,1,0,2,18,49,180,578,1923,3744,2001


In [10]:
# Add the totals.
ds['cases_male_total'] = ds[COLUMNS_CASES_MALE].sum(axis=1).astype(int)
ds['deaths_male_total'] = ds[COLUMNS_DEATHS_MALE].sum(axis=1).astype(int)
ds['cases_female_total'] = ds[COLUMNS_CASES_FEMALE].sum(axis=1).astype(int)
ds['deaths_female_total'] = ds[COLUMNS_DEATHS_FEMALE].sum(axis=1).astype(int)

In [11]:
# Check the transformed table
ds

,cases_male_0-9,cases_male_10-19,cases_male_20-29,cases_male_30-39,cases_male_40-49,cases_male_50-59,cases_male_60-69,cases_male_70-79,cases_male_80-89,cases_male_90-,deaths_male_0-9,deaths_male_10-19,deaths_male_20-29,deaths_male_30-39,deaths_male_40-49,deaths_male_50-59,deaths_male_60-69,deaths_male_70-79,deaths_male_80-89,deaths_male_90-,cases_female_0-9,cases_female_10-19,cases_female_20-29,cases_female_30-39,cases_female_40-49,cases_female_50-59,cases_female_60-69,cases_female_70-79,cases_female_80-89,cases_female_90-,deaths_female_0-9,deaths_female_10-19,deaths_female_20-29,deaths_female_30-39,deaths_female_40-49,deaths_female_50-59,deaths_female_60-69,deaths_female_70-79,deaths_female_80-89,deaths_female_90-,cases_male_total,deaths_male_total,cases_female_total,deaths_female_total
2020-03-12,36,60,214,423,735,1359,1611,1916,1306,155,0,0,0,1,0,10,53,216,252,40,27,57,272,353,717,1033,744,886,794,230,0,0,0,0,1,4,12,57,103,35,7815,572,5113,212
2020-03-16,75,97,419,829,1491,2633,3007,3461,2322,287,0,0,0,4,5,34,113,469,491,81,45,87,517,794,1425,2002,1386,1621,1518,472,0,0,0,0,4,12,29,130,234,84,14621,1197,9867,493
2020-03-23,181,200,983,1988,3601,6276,6943,7458,4930,678,0,0,0,9,31,129,437,1371,1358,208,135,184,1166,1952,3572,4887,3405,3782,3590,1242,0,0,0,3,10,38,99,390,656,253,33238,3543,23915,1449
2020-03-26,244,261,1203,2465,4597,7998,8755,9309,6195,887,0,0,0,14,49,190,606,1846,1808,273,180,249,1510,2494,4570,6337,4394,4781,4734,1640,0,0,0,3,18,52,154,555,894,334,41914,4786,30889,2010
2020-03-30,324,392,1631,3105,5802,10068,10744,11236,7630,1160,0,0,1,18,66,294,923,2597,2603,424,260,371,2132,3352,6198,8495,5584,6142,6504,2404,0,0,1,2,23,74,236,854,1378,514,52092,6926,41442,3082
2020-04-02,381,475,1919,3505,6480,11149,11761,12281,8553,1374,0,0,4,22,81,377,1144,3146,3259,558,307,454,2531,3885,7130,9706,6254,6868,7644,2973,0,0,2,7,29,100,299,1041,1767,692,57878,8591,47752,3937
2020-04-06,450,621,2426,4158,7444,12648,13057,13581,9868,1651,0,0,4,24,102,447,1349,3657,3793,670,375,596,3181,4780,8701,11608,7306,7990,9525,4024,1,0,3,10,34,118,372,1244,2157,854,65904,10046,58086,4793
2020-04-09,509,717,2752,4612,8021,13469,13784,14234,10661,1894,0,0,5,25,114,504,1532,3968,4255,760,425,712,3576,5316,9676,12864,7908,8666,10935,4941,1,0,2,11,39,133,423,1393,2450,1022,70653,11163,65019,5474
2020-04-16,596,901,3350,5344,9009,14779,14963,15577,12332,2470,0,0,5,28,133,606,1776,4532,4992,970,519,897,4338,6299,11452,15015,9024,10097,14329,7333,1,0,2,12,45,150,504,1664,3071,1483,79321,13042,79303,6932
2020-04-23,687,1070,3877,5951,9849,15772,15825,16410,13446,2888,1,0,5,30,154,681,1993,4952,5642,1210,607,1070,5024,7136,12851,16685,9825,11163,17034,9429,1,0,2,18,49,180,578,1923,3744,2001,85775,14668,90824,8496


In [12]:
# Export the transformed table to a csv file.
CSV_FILE_NAME = 'italy_cases_deaths_by_age_sex.csv'
ds.to_csv(EXPORT_DIR+CSV_FILE_NAME, index_label='date')

In [13]:
# Check roundtrip
chk_df = pd.read_csv(EXPORT_DIR+CSV_FILE_NAME, index_col=0, parse_dates=True)
chk_df

,cases_male_0-9,cases_male_10-19,cases_male_20-29,cases_male_30-39,cases_male_40-49,cases_male_50-59,cases_male_60-69,cases_male_70-79,cases_male_80-89,cases_male_90-,deaths_male_0-9,deaths_male_10-19,deaths_male_20-29,deaths_male_30-39,deaths_male_40-49,deaths_male_50-59,deaths_male_60-69,deaths_male_70-79,deaths_male_80-89,deaths_male_90-,cases_female_0-9,cases_female_10-19,cases_female_20-29,cases_female_30-39,cases_female_40-49,cases_female_50-59,cases_female_60-69,cases_female_70-79,cases_female_80-89,cases_female_90-,deaths_female_0-9,deaths_female_10-19,deaths_female_20-29,deaths_female_30-39,deaths_female_40-49,deaths_female_50-59,deaths_female_60-69,deaths_female_70-79,deaths_female_80-89,deaths_female_90-,cases_male_total,deaths_male_total,cases_female_total,deaths_female_total
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-03-12,36,60,214,423,735,1359,1611,1916,1306,155,0,0,0,1,0,10,53,216,252,40,27,57,272,353,717,1033,744,886,794,230,0,0,0,0,1,4,12,57,103,35,7815,572,5113,212
2020-03-16,75,97,419,829,1491,2633,3007,3461,2322,287,0,0,0,4,5,34,113,469,491,81,45,87,517,794,1425,2002,1386,1621,1518,472,0,0,0,0,4,12,29,130,234,84,14621,1197,9867,493
2020-03-23,181,200,983,1988,3601,6276,6943,7458,4930,678,0,0,0,9,31,129,437,1371,1358,208,135,184,1166,1952,3572,4887,3405,3782,3590,1242,0,0,0,3,10,38,99,390,656,253,33238,3543,23915,1449
2020-03-26,244,261,1203,2465,4597,7998,8755,9309,6195,887,0,0,0,14,49,190,606,1846,1808,273,180,249,1510,2494,4570,6337,4394,4781,4734,1640,0,0,0,3,18,52,154,555,894,334,41914,4786,30889,2010
2020-03-30,324,392,1631,3105,5802,10068,10744,11236,7630,1160,0,0,1,18,66,294,923,2597,2603,424,260,371,2132,3352,6198,8495,5584,6142,6504,2404,0,0,1,2,23,74,236,854,1378,514,52092,6926,41442,3082
2020-04-02,381,475,1919,3505,6480,11149,11761,12281,8553,1374,0,0,4,22,81,377,1144,3146,3259,558,307,454,2531,3885,7130,9706,6254,6868,7644,2973,0,0,2,7,29,100,299,1041,1767,692,57878,8591,47752,3937
2020-04-06,450,621,2426,4158,7444,12648,13057,13581,9868,1651,0,0,4,24,102,447,1349,3657,3793,670,375,596,3181,4780,8701,11608,7306,7990,9525,4024,1,0,3,10,34,118,372,1244,2157,854,65904,10046,58086,4793
2020-04-09,509,717,2752,4612,8021,13469,13784,14234,10661,1894,0,0,5,25,114,504,1532,3968,4255,760,425,712,3576,5316,9676,12864,7908,8666,10935,4941,1,0,2,11,39,133,423,1393,2450,1022,70653,11163,65019,5474
2020-04-16,596,901,3350,5344,9009,14779,14963,15577,12332,2470,0,0,5,28,133,606,1776,4532,4992,970,519,897,4338,6299,11452,15015,9024,10097,14329,7333,1,0,2,12,45,150,504,1664,3071,1483,79321,13042,79303,6932
